In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import cv2
import numpy as np
from os import listdir as ld
from os.path import join as pj
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data

# Loader
from IO.loader import load_path, load_images, load_annotations_path, load_annotations, get_anno_recs
# output XML
from IO.utils import output_formatter, write_output_xml
# Dataset
from dataset.detection.dataset import insects_dataset_from_voc_style_txt, collate_fn
# Predict
from model.refinedet.utils.predict import test_prediction
# Evaluate
from evaluation.detection.evaluate import evaluate
# Statistics
from evaluation.detection.statistics import compute_each_size_df, compute_all_size_df, compute_class_df, compute_error_df
# Visualize
from evaluation.detection.visualize import plot_df_distrib_size, plot_pr_curve, plot_df_distrib_class, plot_df_error, vis_detections

import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

# Test Config

In [ ]:
class args:
    # experiment name
    experiment_name = "crop_b2_2_4_8_16_32_im512_CSL"
    # paths
    data_root = "/home/tanida/workspace/Insect_Phenology_Detector/data"
    test_image_root = "/home/tanida/workspace/Insect_Phenology_Detector/data/test_refined_images"
    model_root = pj("/home/tanida/workspace/Insect_Phenology_Detector/output_model/detection/RefineDet", experiment_name)
    figure_root = pj("/home/tanida/workspace/Insect_Phenology_Detector/figure/detection/RefineDet", experiment_name)
    test_anno_folders = ["annotations_4"]
    # training config
    input_size = 512 # choices=[320, 512, 1024]
    crop_num = (5, 5)
    tcb_layer_num = 5
    rm_last = True
    # test config
    save_fig = True
    save_df = True

# Model Config

In [ ]:
if args.tcb_layer_num == 4 and args.rm_last == False:
    from model.refinedet.config import tcb_4_rm_false as insect_refinedet
elif args.tcb_layer_num == 4 and args.rm_last == True:
    from model.refinedet.config import tcb_4_rm_true as insect_refinedet
elif args.tcb_layer_num == 5 and args.rm_last == False:
    from model.refinedet.config import tcb_5_rm_false as insect_refinedet
elif args.tcb_layer_num == 5 and args.rm_last == True:
    from model.refinedet.config import tcb_5_rm_true as insect_refinedet
elif args.tcb_layer_num == 6 and args.rm_last == False:
    from model.refinedet.config import tcb_6_rm_false as insect_refinedet

# Set cuda

In [ ]:
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

# Model

In [ ]:
if args.rm_last == True:
    from model.refinedet.refinedet_rmlast import build_refinedet
else:
    from model.refinedet.refinedet import build_refinedet

### Make data

In [ ]:
print('Loading dataset for test ...')
test_dataset = insects_dataset_from_voc_style_txt(args.test_image_root, args.input_size, args.crop_num, "RefineDet", training=False)
test_data_loader = data.DataLoader(test_dataset, 1, num_workers=1, shuffle=False, collate_fn=collate_fn)
print('Loading annotation for test...')
annos, imgs = load_path(args.data_root, "refined_images", args.test_anno_folders)
images = load_images(imgs)
annotations_path = load_annotations_path(annos, images)
anno = load_annotations(annotations_path)
imagenames, recs = get_anno_recs(anno)

### Get model config

In [ ]:
cfg = insect_refinedet[str(args.input_size)]

### Load model

In [ ]:
print("Loading model for test ...")
model = build_refinedet('test', insect_refinedet, args.input_size, args.tcb_layer_num).cuda()
load_name = pj(args.model_root, 'RefineDet{}_{}.pth'.format(args.input_size, "final"))
model.load_state_dict(torch.load(load_name))

In [ ]:
model

# --- result analysis ---

In [ ]:
result = test_prediction(model, test_data_loader, args.crop_num, nms_thresh=0.3)
recall, precision, avg_precision, gt_dict = evaluate(result, recs)

In [ ]:
avg_precision

In [ ]:
each_label_dic = {
    'Coleoptera': 0,
    'Diptera': 1,
    'Ephemeridae': 2,
    'Ephemeroptera': 3,
    'Hemiptera': 4,
    'Lepidoptera': 5,
    'Plecoptera': 6,
    'Trichoptera': 7,
    'small insect': 8,
    'medium insect': 9
}

### --- visualize accuracy distribution of size ---

In [ ]:
if os.path.exists(args.figure_root) is False:
    os.makedirs(args.figure_root)

In [ ]:
each_df = compute_each_size_df(gt_dict)
if args.save_df is True:
    each_df.to_csv(pj(args.figure_root, "each_size_df.csv"))
each_df

In [ ]:
all_df = compute_all_size_df(each_df)
if args.save_df is True:
    all_df.to_csv(pj(args.figure_root, "all_size_df.csv"))
all_df

In [ ]:
plot_df_distrib_size(all_df, args.figure_root, save=args.save_fig)

In [ ]:
df = pd.DataFrame({"precision": precision, "recall": recall})
if args.save_df is True:
    df.to_csv(pj(args.figure_root, "pr_curve_df.csv"))
df

In [ ]:
plot_pr_curve(df, args.figure_root, save=args.save_fig)

### --- visualize accuracy distribution of class ---

In [ ]:
df = compute_class_df(each_label_dic, gt_dict)
if args.save_df is True:
    df.to_csv(pj(args.figure_root, "each_class_df.csv"))
df

In [ ]:
plot_df_distrib_class(df, args.figure_root, save=args.save_fig, color="green")

### --- visualize error count per class ---

In [ ]:
df = compute_error_df(each_label_dic, gt_dict)
if args.save_df is True:
    df.to_csv(pj(args.figure_root, "each_class_error_df.csv"))
df

In [ ]:
plot_df_error(df, args.figure_root, save=args.save_fig, color="green")

### --- output labelImg XML ---

In [ ]:
output = output_formatter(result, thresh=0.3)
write_output_xml(output, "/home/tanida/workspace/Insect_Phenology_Detector/output_xml/refinedet_crop_b2_2_4_8_16_32_im512")

### --- compare ground truth and output ---

In [ ]:
im_index = 0
if os.path.exists(args.figure_root) is False:
    os.makedirs(args.figure_root)

In [ ]:
gt   = np.asarray(list(map(lambda x:x["bbox"]+[1], recs[imagenames[im_index]])))
gazo = np.asarray(Image.open(pj("/home/tanida/workspace/Insect_Phenology_Detector/data/refined_images", imagenames[im_index]+".png")))
x = vis_detections(gazo, result[imagenames[im_index]], color_name="blue")
x = vis_detections(x, gt, color_name="red")
x = Image.fromarray(x)
x.save(pj(args.figure_root, imagenames[im_index]+".png"))
print(imagenames[im_index])
im_index += 1
#x

### --- compare different iou ---

In [ ]:
for thresh in [.3, .5, .7]:
    recall, precision, avg_precision, gt_dict = evaluate(result, recs, ovthresh=thresh)
    print("thresh == {0}, ap == {1}".format(thresh, avg_precision))